In [1]:
import json, pandas as pd
from IPython.display import display, Markdown

In [2]:
experiments = {
    "gpt-3.5-turbo": json.load(open('experiments/example/gpt-3.5-turbo-owl-inf.json', 'r')),
    "gpt-4o-2024-05-13": json.load(open('experiments/example/gpt-4o-2024-05-13-owl-inf.json', 'r')),
    "gpt-4-0125-preview": json.load(open('experiments/example/gpt-4-0125-preview-owl-inf.json', 'r')),
    "gpt-4o-mini-2024-07-18": json.load(open('experiments/example/gpt-4o-mini-2024-07-18-owl-inf.json', 'r')),
    "mistralai/Mistral-7B-Instruct-v0.3": json.load(open('experiments/example/Mistral-7B-Instruct-v0.3-owl-inf.json', 'r')),
    "mistralai/Mixtral-8x7B-Instruct-v0.1": json.load(open('experiments/example/Mixtral-8x7B-Instruct-v0.1-owl-inf.json', 'r')),
    "meta-llama/Meta-Llama-3-70b-Instruct": json.load(open('experiments/example/Meta-Llama-3-70B-Instruct-owl-inf.json', 'r')),
    "claude-3-opus-20240229": json.load(open('experiments/example/claude-3-opus-20240229-owl-inf.json', 'r')),
    "claude-3-5-sonnet-20240620": json.load(open('experiments/example/claude-3-5-sonnet-20240620-owl-inf.json', 'r')),
    "claude-3-haiku-20240307": json.load(open('experiments/example/claude-3-haiku-20240307-owl-inf.json', 'r')),
}

In [3]:
dfs = [ pd.DataFrame.from_records(triples) for triples in [ experiments[model] for model in experiments ] ]
df = pd.concat(dfs)

In [4]:
# Convert 'predicted' column from string to integer, replacing errors with NaN
df['answer'] = pd.to_numeric(df['answer'], errors='coerce')

# Drop any rows where 'predicted' is now NaN
df = df.dropna(subset=['answer'])

# Convert 'predicted' to integer type
df['answer'] = df['answer'].astype(int)

# Now perform the groupby and sum operation
result = df.groupby('model')['answer'].sum().reset_index()

# Calculate 'N'
result['FN'] = len(dfs[0]) - result['answer']

# Calculate false negative rate (FNR)
result["FNR"] = (len(dfs[0]) - result["answer"]) / len(dfs[0])

# Display result
result[['model', 'FN', 'FNR']].sort_values('FNR', ascending=True)

,model,FN,FNR
0,claude-3-5-sonnet-20240620,0,0.000
1,claude-3-haiku-20240307,0,0.000
4,gpt-4-0125-preview,0,0.000
5,gpt-4o-2024-05-13,0,0.000
7,meta-llama/Meta-Llama-3-70B-Instruct,0,0.000
2,claude-3-opus-20240229,1,0.125
6,gpt-4o-mini-2024-07-18,1,0.125
3,gpt-3.5-turbo,3,0.375
9,mistralai/Mixtral-8x7B-Instruct-v0.1,3,0.375
8,mistralai/Mistral-7B-Instruct-v0.3,4,0.500


In [5]:
for model in experiments:
    display(Markdown(f'## Model: {model}'))
    df2 = pd.DataFrame.from_records(experiments[model])[['s', 'p', 'o', 'answer', 'rationale']].sort_values('s')
    fns_styler = df2.style.set_properties(**{"text-align": "left", "vertical-align" : "top", "overflow-wrap": "break-word"}).hide(axis="index")
    display(fns_styler)

## Model: gpt-3.5-turbo

s,p,o,answer,rationale
http://example.org/x1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/t1,0,"In the given knowledge graph, the subject ex:x1 is related to ex:y1 and ex:y2 through the predicate ex:r2. The predicate ex:r2 has a domain of ex:t1 and a range of ex:t2, and it is defined as an owl:FunctionalProperty. However, there is no direct connection between ex:x1 and ex:t1 in the provided information. Therefore, the triple is false in this hypothetical world."
http://example.org/x1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/x1,1,"In the given knowledge graph, the triple states that the subject is the same as the object . In RDF, the owl:sameAs predicate is used to indicate that two URI references actually refer to the same thing. Therefore, in this hypothetical world, where the same URI reference is used for both subject and object, the triple is true."
http://example.org/y1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y1,1,"In the given knowledge graph, the triple states that the subject is the same as the object . In RDF, the owl:sameAs predicate is used to indicate that two URI references actually refer to the same thing. Therefore, in this hypothetical world, where the same URI reference is used for both subject and object, the triple is true."
http://example.org/y1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/t2,1,"Based on the provided ontology, we can see that the predicate ex:r2 has a range of ex:t2. This means that any subject that has the predicate ex:r2 should have an object of type ex:t2. Since the triple in question has the object , which matches the range of the predicate, the triple is true."
http://example.org/y1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y2,0,"In the given knowledge graph, there are two triples stating that ex:x1 is related to both ex:y1 and ex:y2 through the predicate ex:r2. Since ex:r2 is defined as a FunctionalProperty in the ontology, it implies that each subject can only have one unique object for this predicate. Therefore, the triple is false in this hypothetical world because ex:y1 and ex:y2 cannot be the same entity if they are related to ex:x1 through the same functional property."
http://example.org/y2,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y2,1,"In the given knowledge graph, the triple states that the subject is the same as the object . In RDF, the owl:sameAs predicate is used to indicate that two URI references actually refer to the same thing. Therefore, in this hypothetical world, where the same URI is used for both subject and object, the triple is true."
http://example.org/y2,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y1,0,"In the provided knowledge graph, it is stated that ex:y1 and ex:y2 are both related to ex:x1 through the predicate ex:r2. Additionally, ex:r2 is defined as a FunctionalProperty with a domain of ex:t1 and a range of ex:t2. However, there is no explicit statement in the graph that ex:y1 and ex:y2 are the same entity. Therefore, the triple is false in this hypothetical world."
http://example.org/y2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/t2,1,"Based on the provided ontology, we can see that the predicate ex:r2 has a domain of ex:t1 and a range of ex:t2. Since the object in the triple is , which matches the range of the predicate, the triple is true."


## Model: gpt-4o-2024-05-13

s,p,o,answer,rationale
http://example.org/x1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/t1,1,"1. **Understanding the Triple**: - Subject: `` - Predicate: `` - Object: `` This triple asserts that `` is of type ``. 2. **Analyzing the Given Information**: - `ex:x1 ex:r2 ex:y1 .` - `ex:x1 ex:r2 ex:y2 .` - `ex:r2 rdfs:domain ex:t1 .` - `ex:r2 rdfs:range ex:t2 .` - `ex:r2 rdf:type owl:FunctionalProperty .` 3. **Implications of the Given Information**: - `ex:r2 rdfs:domain ex:t1` means that any subject of the property `ex:r2` must be of type `ex:t1`. - `ex:r2 rdfs:range ex:t2` means that any object of the property `ex:r2` must be of type `ex:t2`. - `ex:r2 rdf:type owl:FunctionalProperty` means that `ex:r2` is a functional property, implying that for a given subject, there can be at most one unique object. 4. **Checking the Subject**: - `ex:x1 ex:r2 ex:y1` and `ex:x1 ex:r2 ex:y2` both indicate that `ex:x1` is the subject of the property `ex:r2`. - Given `ex:r2 rdfs:domain ex:t1`, `ex:x1` must be of type `ex:t1`. 5. **Consistency Check**: - The functional property `ex:r2` implies that `ex:x1` should have only one unique object for `ex:r2`. However, `ex:x1` has two objects (`ex:y1` and `ex:y2`), which seems contradictory. But this does not affect the type assertion of `ex:x1`. 6. **Conclusion**: - Despite the apparent contradiction in the functional property, the type assertion `ex:x1 rdf:type ex:t1` is directly supported by the domain declaration of `ex:r2`. ###"
http://example.org/x1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/x1,1,"1. **Understanding the Triple**: - The triple in question is: ``` ``` - This triple asserts that the subject `` is the same as the object ``. 2. **Understanding `owl:sameAs`**: - The `owl:sameAs` predicate is used to state that two URI references actually refer to the same thing. In this case, it is stating that `` is the same as itself. 3. **Analyzing the Given Information**: - The provided RDF data includes: ``` ex:x1 ex:r2 ex:y1 . ex:x1 ex:r2 ex:y2 . ex:r2 rdfs:domain ex:t1 . ex:r2 rdfs:range ex:t2 . ex:r2 rdf:type owl:FunctionalProperty . ``` - The predicate `ex:r2` is defined as a functional property (`owl:FunctionalProperty`), meaning that for a given subject, there can be at most one unique object for this property. 4. **Implications of Functional Property**: - Given that `ex:r2` is a functional property, the statements: ``` ex:x1 ex:r2 ex:y1 . ex:x1 ex:r2 ex:y2 . ``` imply that `ex:y1` and `ex:y2` must be the same individual because a functional property can only have one unique object for a given subject. 5. **Relevance to `owl:sameAs`**: - The functional property constraint and the domain and range constraints do not directly affect the truth value of the `owl:sameAs` triple in question. - The triple ` ` is trivially true because any entity is always the same as itself. ###"
http://example.org/y1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y1,1,"To determine the truth value of the triple: ``` Subject: Predicate: Object: ``` we need to understand the semantics of the `owl:sameAs` predicate and the provided RDF data. 1. **Understanding `owl:sameAs`:** - The `owl:sameAs` predicate is used to state that two URI references actually refer to the same thing. In this case, it is asserting that `` is the same as ``. - This is trivially true because any entity is always the same as itself. 2. **Analyzing the provided RDF data:** - The RDF data provided includes information about properties and their domains and ranges, but it does not directly affect the truth value of the `owl:sameAs` statement for `` and itself. - The data states: ``` ex:x1 ex:r2 ex:y1 . ex:x1 ex:r2 ex:y2 . ex:r2 rdfs:domain ex:t1 . ex:r2 rdfs:range ex:t2 . ex:r2 rdf:type owl:FunctionalProperty . ``` - These statements define relationships and constraints involving `ex:r2`, `ex:y1`, and `ex:y2`, but they do not contradict the self-identity of `ex:y1`. 3. **Conclusion:** - Since `owl:sameAs`

## Model: gpt-4-0125-preview

...

s,p,o,answer,rationale
http://example.org/x1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/t1,1,"The rdfs:domain of ex:r2 is stated to be ex:t1. ex:x1 is the subject of triples with predicate ex:r2. Therefore, ex:x1 must be of type ex:t1, making the triple ex:x1 rdf:type ex:t1 true."
http://example.org/x1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/x1,1,"The owl:sameAs property is reflexive, meaning every individual is the same as itself. The provided triples do not contain any information that would contradict this statement for the individual ex:x1."
http://example.org/y1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y1,1,"The owl:sameAs property is reflexive, meaning everything is the same as itself. The other triples provided do not contradict this."
http://example.org/y1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/t2,1,"The given triple ex:y1 rdf:type ex:t2 is true because ex:x1 has relation ex:r2 to ex:y1, and the range of ex:r2 is stated to be the class ex:t2. In RDFS semantics, this implies that ex:y1 is an instance of class ex:t2."
http://example.org/y1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y2,1,"Since ex:r2 is a functional property, and ex:x1 has the relation ex:r2 to both ex:y1 and ex:y2, it follows that ex:y1 and ex:y2 must refer to the same entity. Therefore, the triple stating that they are the same using owl:sameAs is true."
http://example.org/y2,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y2,1,"The triple asserts that ex:y2 is the same as itself, which is always true. The functional property constraint on ex:r2 does not affect the truth of this triple."
http://example.org/y2,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y1,1,"Since ex:r2 is an owl:FunctionalProperty, ex:y1 and ex:y2 must be the same entity (owl:sameAs) given that ex:x1 has ex:r2 relations to both. Therefore, the triple stating ex:y2 owl:sameAs ex:y1 must be true."
http://example.org/y2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/t2,0,The given facts lead to an inconsistency due to the violation of the owl:FunctionalProperty constraint. This inconsistency means we cannot reliably infer the type of ex:y2.


## Model: claude-3-5-sonnet-20240620

s,p,o,answer,rationale
http://example.org/x1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/t1,1,"The triple can be inferred from the given information using the rdfs:domain property of ex:r2, which implies that any subject of ex:r2 (in this case, ex:x1) must be of type ex:t1."
http://example.org/x1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/x1,1,"The owl:sameAs predicate is reflexive, meaning that any entity is always the same as itself. Since both the subject and object of the triple are identical (), this triple is true by definition, regardless of any other information provided about x1."
http://example.org/y1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y1,1,"The owl:sameAs predicate expresses identity, and any entity is always identical to itself. The triple owl:sameAs is a statement of self-identity, which is always true regardless of any other information in the knowledge base."
http://example.org/y1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/t2,1,"The triple is true because ex:y1 is the object of a triple with predicate ex:r2, and ex:r2 has a range of ex:t2. According to RDF semantics, this implies that ex:y1 must be of type ex:t2."
http://example.org/y1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y2,1,"Given that ex:r2 is a functional property and ex:x1 has both ex:y1 and ex:y2 as values for this property, we can conclude that ex:y1 and ex:y2 must refer to the same entity. This is exactly what the owl:sameAs predicate expresses."
http://example.org/y2,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y2,1,"The owl:sameAs predicate expresses identity, and any entity is always identical to itself. This is a fundamental logical principle that holds regardless of any other contextual information."
http://example.org/y2,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y1,1,"The functional property ex:r2 relates ex:x1 to both ex:y1 and ex:y2. Since a functional property can only relate a subject to one unique object, this implies that ex:y1 and ex:y2 must be the same entity. Therefore, the statement ex:y2 owl:sameAs ex:y1 is true."
http://example.org/y2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/t2,1,"The triple is true because y2 is the object of relation r2, and r2 has a range of t2. According to RDFS semantics, this implies that y2 is of type t2."


## Model: claude-3-haiku-20240307

s,p,o,answer,rationale
http://example.org/x1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/t1,1,"The given knowledge graph triple states that the subject is of type . To determine the truth value of this triple, we need to examine the information provided in the hypothetical world. In the hypothetical world, we have the following information: 1. is related to and through the property . 2. The property has a domain of and a range of . 3. The property is a functional property, meaning that for a given subject, there can be at most one object related to it through this property. Based on this information, we can conclude that the subject is of type , as the property has a domain of . Therefore, the given knowledge graph triple is true."
http://example.org/x1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/x1,1,"The given knowledge graph triple states that the resource `` is the same as itself, i.e., `` is equivalent to ``. This is a true statement, as any resource is always the same as itself. In the hypothetical world where the additional information is provided, we can see that: 1. `ex:x1` is related to `ex:y1` and `ex:y2` through the property `ex:r2`. 2. `ex:r2` is a functional property, meaning that for any given subject, there can be at most one object related to it through `ex:r2`. 3. `ex:r2` has a domain of `ex:t1` and a range of `ex:t2`, which means that the subject and object of `ex:r2` must be instances of `ex:t1` and `ex:t2`, respectively. Given this information, the statement ` ` is a true statement, as any resource is always the same as itself."
http://example.org/y1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y1,1,"The given knowledge graph triple states that the resource `` is the same as itself, i.e., `` is equivalent to ``. This is a true statement, as any resource is always the same as itself. The additional information provided in the hypothetical world further supports this. The triple `ex:x1 ex:r2 ex:y1 .` indicates that the resource `ex:y1` is related to `ex:x1` through the property `ex:r2`. The triple `ex:x1 ex:r2 ex:y2 .` indicates that the resource `ex:y2` is also related to `ex:x1` through the same property `ex:r2`. The property `ex:r2` is also defined as a functional property, meaning that for a given subject, there can be at most one object related to it through this property. Given this information, the statement `` is the same as itself is a true statement, as any resource is always equivalent to itself."
http://example.org/y1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/t2,1,The given knowledge graph triple is true based on the information provided in the hypothetical world.
http://example.org/y1,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y2,1,"The given knowledge graph triple states that the resource `` is the same as the resource ``, as indicated by the `` predicate. In the hypothetical world described by the provided RDF triples, we have the following information: 1. `ex:x1 ex:r2 ex:y1` and `ex:x1 ex:r2 ex:y2` - This means that the resources `ex:y1` and `ex:y2` are both related to `ex:x1` via the `ex:r2` property. 2. `ex:r2 rdfs:domain ex:t1` and `ex:r2 rdfs:range ex:t2` - This means that the `ex:r2` property has a domain of `ex:t1` and a range of `ex:t2`. 3. `ex:r2 rdf:type owl:FunctionalProperty` - This means that the `ex:r2` property is a functional property, which implies that for a given subject, there can be at most one object related to it via the `ex:r2` property. Given this information, we can conclude that the resources `ex:y1` and `ex:y2` are the same, as they are both related to the same subject `ex:x1` via the functional property `ex:r2`. Therefore, the knowledge graph triple ` ` is true in the hypothetical world described by the provided RDF triples."
http://example.org/y2,http://www.w3.org/2002/07/owl#sameAs,http://example.org/y2,1,"The given knowledge graph triple states that the resource `` is the same as